In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import seaborn as sns
import spacy

from gensim.models import Word2Vec
from scapy.all import PcapReader
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from read_pcaps import pcap_to_dataframe

2024-02-02 10:47:30.707956: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-02 10:47:30.768527: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-02 10:47:30.768583: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-02 10:47:30.770429: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-02 10:47:30.780058: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-02 10:47:32.374150: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

In [2]:
# define a variable that allows you to read prior saved pkl files
READ_FROM_PKL = True

In [3]:
if READ_FROM_PKL:
    mirai_df = pd.read_pickle("../data/blog_eda/mirai.pkl")
    benign_df = pd.read_pickle("../data/blog_eda/benign.pkl")
else:
    pcap_reader_mirai = PcapReader("../data/blog_eda/mirai.pcap")
    pcap_reader_benign = PcapReader("../data/blog_eda/benign.pcapng")
    mirai_df = pcap_to_dataframe(pcap_reader_mirai)
    benign_df = pcap_to_dataframe(pcap_reader_benign)

In [4]:
# read features saved, add new ones
if READ_FROM_PKL:
    mirai_features_df = pd.read_pickle("../data/blog_fe/mirai_features.pkl")
    benign_features_df = pd.read_pickle("../data/blog_fe/benign_features.pkl")
else:
    print(
        "Error! Feature pkl not saved. Please run blog_fe_2.ipynb, blog_fe_3.ipynb, blog_fe_4.ipynb or download from here: https://drive.google.com/drive/folders/1dBQhbQtIk_fbbb80G5pSVV3hbWYJY7fv?usp=sharing"
    )

In [5]:
# drop rows with None
mirai_df = mirai_df.dropna()
benign_df = benign_df.dropna()

# Add labels

In [6]:
# add labels, 0 for benign, 1 for malicious
mirai_features_df["Label"] = 1
benign_features_df["Label"] = 0

/tmp/ipykernel_525770/2179463880.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mirai_features_df["Label"] = 1
/tmp/ipykernel_525770/2179463880.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  benign_features_df["Label"] = 0


In [7]:
concatenated_df = pd.concat([mirai_features_df, benign_features_df], ignore_index=True)

In [8]:
shuffled_df = shuffle(concatenated_df, random_state=42)

In [9]:
# make sure that all features are numeric, otherwise feature selection will not work
shuffled_df.columns

Index(['Timestamp', 'Source Port', 'Destination Port', 'Packet Length',
       'Protocol', 'incoming_traffic', 'outgoing_traffic', 'traffic_ratio',
       'cumulative_length', 'interarrival',
       ...
       'SNMP_trap', 'HTTPS', 'Syslog', 'LDAPS', 'FTPS', 'IMAPS', 'POP3S',
       'SOCKS_proxy', 'payload_embedding', 'Label'],
      dtype='object', length=126)

# Split data
- Train
- Test

In [10]:
# X should contain your feature columns, and y should contain the labels (0 or 1)
X = shuffled_df.drop("Label", axis=1)
y = shuffled_df["Label"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Filter

In [11]:
# Select the top k features using SelectKBest and f_classif
k_best = 5  # You can adjust this value based on your dataset and requirements
selector = SelectKBest(f_classif, k=k_best)
X_train_selected = selector.fit_transform(X_train, y_train)

# Train a classifier using the selected features
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_selected, y_train)

# Transform the test set using the same feature selector
X_test_selected = selector.transform(X_test)

# Make predictions and evaluate the performance
y_pred = clf.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)

print(f"Selected {k_best} features using SelectKBest and f_classif.")
print(f"Accuracy on the test set: {accuracy}")

ValueError: could not convert string to float: 'LOW'

# Wrapper

# PCA